In [1]:
from osgeo import ogr
import numpy as np
import cv2

path = "../nautical_chart/US3CA14M/ENC_ROOT/US3CA14M/"
file_name = "US3CA14M.000"

# target_layer_list = ["BOYLAT", "COALNE", "DEPCNT", "DEPARE", "SOUNDG", "LNDARE"]
target_layer_list = ["LNDARE"]
layer_dict = {}
extent = [180, -180, 90, -90]  # initialization of extent: [longitude_min, longitude_max, latitude_min, latitude_max]
chart_size = []
chart = None

resolution = 100  # pixels per km, distance calculated approximately at (37.84, -122.40)
x_km_per_degree = 87.81  # x for longitude
y_km_per_degree = 111.19  # y for latitude

def geo_to_pixel(extent, chart_size, geo_position):
    # chart_size[0]: latitude_range
    # chart_size[1]: longitude_range
    pos_in_pixel = []
    for i in range(len(geo_position)):
        x_geo = geo_position[i][0]
        y_geo = geo_position[i][1]
        x_geo_range = extent[1] - extent[0]
        y_geo_range = extent[3] - extent[2]
        x_pixel = int((x_geo - extent[0]) / (x_geo_range / chart_size[1]))
        y_pixel = int((y_geo - extent[2]) / (y_geo_range / chart_size[0]))
#         pos_in_pixel.append((y_pixel, x_pixel))
        pos_in_pixel.append((x_pixel, y_pixel))
    return pos_in_pixel
  
def get_layer_by_name(dataSource, name):
    layer = dataSource.GetLayerByName(name)
    if layer:
#         print("Has layer: ", name)
        return layer
    else:
        print("Do not contain " + name)
        return None
   

def update_extent(original_extent, new_extent):
    og = original_extent
    new = new_extent
    if new[0] < og[0]: og[0] = new[0]
    if new[1] > og[1]: og[1] = new[1]
    if new[2] < og[2]: og[2] = new[2]
    if new[3] > og[3]: og[3] = new[3]  
    return og



In [ ]:
driver = ogr.GetDriverByName('S57')
dataSource = driver.Open(path+file_name, 0)

# get all target layers which exist
for layer_name in target_layer_list:
    layer = get_layer_by_name(dataSource, layer_name)
    if layer:
        layer_dict[layer_name] = layer

# get the extent of the chart
for key in layer_dict:
    layer_extent = layer_dict[key].GetExtent()
    extent = update_extent(extent, layer_extent)
print("The extent of the chart: ", extent)

# initialize the chart in pixel coordinate
# chart_size = (int((extent[1] - extent[0]) * x_km_per_degree * resolution), int((extent[3] - extent[2]) * y_km_per_degree * resolution))
chart_size = (int((extent[3] - extent[2]) * y_km_per_degree * resolution), \
              int((extent[1] - extent[0]) * x_km_per_degree * resolution))

print("Chart size in pixel (lat, lon):", chart_size)
chart = np.zeros(chart_size)

# get all land areas
land_area_layer = get_layer_by_name(dataSource, "LNDARE")
num_land_area_features = land_area_layer.GetFeatureCount()
for i in range(num_land_area_features):
    land_area_feature = land_area_layer.GetNextFeature()
    land_area = land_area_feature.GetGeometryRef()  # POLYGON

    for j in range(land_area.GetGeometryCount()):  # LINEARRING
        ring = land_area.GetGeometryRef(j)
        ring_points_geo = ring.GetPoints()
        ring_points_pixel = geo_to_pixel(extent, chart_size, ring_points_geo)
        ring_contour = np.array(ring_points_pixel)

        area_chart = np.zeros(chart_size)
        cv2.fillPoly(area_chart, pts=[ring_contour], color=255)
        chart = np.maximum(chart, area_chart)

The extent of the chart:  [-124.6150263, -122.186738, 37.5099574, 39.0748544]
Chart size in pixel (lat, lon): (17400, 21322)


In [3]:
new_shape = (chart.shape[0] // 5, chart.shape[1] // 5)
img = cv2.resize(chart, [new_shape[1], new_shape[0]])

# img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
img = cv2.flip(img, 0)
cv2.imshow('chart', img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [4]:
# interested range: (-122.6666667, 37.5099574) to (-122.186738, 38.1666667)
lon_range = (-122.67, -122.22)
lat_range = (37.54, 38.17)

x0, y0 = geo_to_pixel(extent, chart_size, [[lon_range[0], lat_range[0]]])[0]
x1, y1 = geo_to_pixel(extent, chart_size, [[lon_range[1], lat_range[1]]])[0]

interested_area = chart[y0:y1, x0:x1]
new_shape = (interested_area.shape[0] // 10, interested_area.shape[1] // 10)

img = cv2.resize(interested_area, [new_shape[1], new_shape[0]])
img = cv2.flip(img, 0)

cv2.imshow('chart', img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [107]:
save_path = "../nautical_chart/"
fig_name = save_path + "res_" + str(resolution) + "_lon_" + \
           str(lon_range[0]) + "_" + str(lon_range[1]) + "_lat_" + \
           str(lat_range[0]) + "_" + str(lat_range[1]) + ".jpg"
cv2.imwrite(fig_name, cv2.flip(interested_area, 0))

txt_name = save_path + "chart_detail.txt"
with open(txt_name, 'w') as f:
    f.write("pixels per km: " + str(resolution) + "\n")
    f.write("longitude range: " + str(lon_range) + "\n")
    f.write("latitude range: " + str(lat_range))

import pandas as pd
df_descript = pd.DataFrame([(resolution, x_km_per_degree, y_km_per_degree, lon_range[0], lon_range[1], lat_range[0], lat_range[1])], \
                           columns=["res", "x_km_per_deg", "y_km_per_deg", "lon_0", "lon_1", "lat_0", "lat_1"])
csv_name = save_path + "chart_detail.csv"
df_descript.to_csv(csv_name, index=False)